# Filters
The Google Charts API permits the creation of a dashboard with controls, which allows the user to filter the data displayed in the chart.  JoogleChart supports the addition of one or more filters to a chart.

A filter can "bind" the data or another filter.  When a filter binds the data, the filter affects only the data displayed.  When a filter binds another filter, it affects both the data displayed and the filtering options in the other filter.  By default, a filter in JoogleChart binds the data.  To bind another filter, the binding must be explicitly set.

### Filter types
Google Charts currently offers the following filters.  All types are supported in JoogleCharts.

- CategoryFilter
- NumberRangeFilter
- DateRangeFilter
- ChartRangeFilter
- StringFilter

### Creating a filter
1.  Create a filter object
2.  Add filter options, as found in the Google Charts documenation.
3.  Add the filter to the chart
4.  Bind another filter, if applicable.

### Setting filter state
We can initialize the filter control with state properties if we want.  See the Google Charts documentation for each control's state properties.

In [23]:
import pandas as pd
from jooglechart_api import JoogleChart, Filter

## CategoryFilter

In [24]:
list1 = []
list1.append(['Salesperson', 'State', 'Sales'])
list1.append(['Bobby', 'TX', 34])
list1.append(['Bobby', 'LA', 23])
list1.append(['Bobby', 'CO', 84])
list1.append(['Marsha', 'TX', 62])
list1.append(['Marsha', 'IL', 46])
list1.append(['Marsha', 'OK', 68])
list1.append(['Cindy', 'KY', 24])
list1.append(['Cindy', 'LA', 86])
list1.append(['Cindy', 'OK', 45])

table = JoogleChart(list1, chart_type="Table")

# create the filter object
filter1 = Filter(type="CategoryFilter")

# set the filter options
filter1.add_options(filterColumnIndex=0, ui_allowMultiple=True, ui_allowNone=True)

table.add_filter(filter1)

table.show()


## CategoryFilter: binding another filter

In [25]:
filter2 = Filter(type="CategoryFilter")
filter2.add_options(filterColumnIndex=1, ui_allowMultiple=True, ui_allowNone=True)

table.add_filter(filter2)

filter1.bind_filter(filter2)

table.show()

## NumberRangeFilter

In [26]:
cities = ['New York', 'Chicago', 'Dallas', 'Boston']
pies = [12, 32, 5, 20]
pies_df = pd. DataFrame({'cities': cities, 'pies': pies}, columns=['cities', 'pies'])

pie = JoogleChart(pies_df, chart_type="PieChart")

filter1 = Filter(type="NumberRangeFilter")
filter1.add_options(filterColumnIndex=1)

pie.add_filter(filter1)

pie.show()


## NumberRangeFilter:  adding state

In [27]:
# We will add state values to the filter we created above.  Notice that we can modify the filter object
# at any time, even after it has been added to the chart object.
filter1.add_state(lowValue=10, highValue=25)

pie.show()